# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [24]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [25]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [26]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table' # ENTER A TABLE COLUMNS HERE,
       , 'definition' # ENTER A TABLE DEFINITATIONS HERE
        }
df = pd.DataFrame(data)
print(df)

            0
0  definition
1       table


In [29]:
import pandas as pd

# Corrected table and definitions data
data = {
    'table': ['employees', 'salary', 'studies'],
    'definition': [
        'Employee information, name, position, etc.',
        'Salary details for each year',
        'Educational studies, name of the institution, type of studies, level'
    ]
}

df = pd.DataFrame(data)
print(df)


       table                                         definition
0  employees         Employee information, name, position, etc.
1     salary                       Salary details for each year
2    studies  Educational studies, name of the institution, ...


In [34]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [35]:
print(text_tables)

employees: Employee information, name, position, etc.
salary: Salary details for each year
studies: Educational studies, name of the institution, type of studies, level


In [ ]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Question:
{question}
"""

# queries
user_query_1 = "Which employees have salaries above $100,000?"
user_query_2 = "What is the educational background of employees?"



In [45]:
 #  prompt with the first query
pqt1 = prompt_question_tables.format(tables=text_tables, question=user_query_1)
print("Prompt 1:\n", pqt1)




Prompt 1:
 
Given the following tables and their content definitions,
###Tables
employees: Employee information, name, position, etc.
salary: Salary details for each year
studies: Educational studies, name of the institution, type of studies, level

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
Which employees have salaries above $100,000?



In [46]:
print(return_OAI(pqt1))

```json
{
    "tables": ["employees", "salary"]
}
```


In [47]:
#  prompt with the second query
pqt2 = prompt_question_tables.format(tables=text_tables, question=user_query_2)
print("Prompt 2:\n", pqt2)

Prompt 2:
 
Given the following tables and their content definitions,
###Tables
employees: Employee information, name, position, etc.
salary: Salary details for each year
studies: Educational studies, name of the institution, type of studies, level

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
What is the educational background of employees?



In [48]:
print(return_OAI(pqt2))

```json
{
    "tables": ["employees", "studies"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [52]:
user_query_3 = "Provide a summary of employees, their salaries, and educational qualifications."

In [53]:
#  prompt with the third query

pqt3 = prompt_question_tables.format(tables=text_tables, question=user_query_3)
print("Prompt 3:\n", pqt3)


Prompt 3:
 
Given the following tables and their content definitions,
###Tables
employees: Employee information, name, position, etc.
salary: Salary details for each year
studies: Educational studies, name of the institution, type of studies, level

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
Provide a summary of employees, their salaries, and educational qualifications.



In [54]:
print(return_OAI(pqt2))

{
    "tables": ["employees", "studies"]
}


On gthe third prompt, It seems the model's response missed this crucial element, indicating it could benefit from further refinement of the prompt.
The employees table provides general information about employees.
The salary table is necessary to provide details about employee salaries.
The studies table gives information on their educational qualifications.
Therefore, including salary alongside employees and studies is necessary for a complete answer to the query "Provide a summary of employees, their salaries, and educational qualifications.

In [ ]:
refined_prompt = """
Given the following tables and their content definitions:
### Tables
{tables}

You are tasked with identifying the tables necessary to create an SQL query to answer the user's question below. The response should ensure that all relevant tables needed to fully address the query are included.

Return the table names in a JSON format.
### User Question:
{question}

### Important Considerations:
- Ensure that tables related to salaries, employee details, and education (if mentioned in the query) are included.
- Only exclude tables that have no direct relevance to the query.
"""

# Applying the refined prompt to the query
pqt_refined = refined_prompt.format(tables=text_tables, question=user_query_4)
print("Refined Prompt:\n", pqt_refined)


Refined Prompt:
 
Given the following tables and their content definitions:
### Tables
employees: Employee information, name, position, etc.
salary: Salary details for each year
studies: Educational studies, name of the institution, type of studies, level

You are tasked with identifying the tables necessary to create an SQL query to answer the user's question below. The response should ensure that all relevant tables needed to fully address the query are included.

Return the table names in a JSON format.
### User Question:
Provide a summary of employees, their salaries, and educational qualifications.

### Important Considerations:
- Ensure that tables related to salaries, employee details, and education (if mentioned in the query) are included.
- Only exclude tables that have no direct relevance to the query.



In [60]:
print(return_OAI(pqt_refined))

{
    "tables": ["employees", "salary", "studies"]
}


Final observation after refining the prompt

1. The prompt was refined by emphasizing that all relevant tables should be included, with examples provided such as "tables related to salaries, employee details, and education" to guide the model toward a more accurate response.

2.  The revised prompt specified that the objective was to create an SQL query, emphasizing the inclusion of all relevant tables, ensuring the model provided a comprehensive response.

3. : Clearer instructions were given to minimize the likelihood of important tables being missed or overlooked, resulting in more accurate and complete responses from the model.





